<a href="https://colab.research.google.com/github/cfcastillo/DS-6-Notebooks/blob/main/1_Education_Capstone_Data_Collection_and_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Education Capstone - Data Collection and Cleaning

# Project Definition

The purpose of this project is to identify what factors influence people to choose certain professions or trades. In understanding these factors, we can help colleges like Central New Mexico College (CNM) offer courses that support those professions and better target their marketing to people who are likely to choose those professions.

This project will be a supervised classification problem using tree-based models to identify the factors that will contribute to career choice.



# Data Identification Process

Steps:

1. We stated several questions we wanted answered (target). 
1. After defining our problem, we listed sets of variables that we believed could answer our questions. We then put the variables and targets into a [spreadsheet](https://docs.google.com/spreadsheets/d/1bOhOBHKOae9TDN9n9-xF7ag4QW_Z0c7HXTYLXeMMLHs/edit#gid=0) to define the dataset we would need to run our analysis. 
1. We then researched data sources such as Bureau of Labor Statistics and the US Census to locate data that supported our research. 
1. We mapped the columns in the data sources to the columns in our desired dataset and linked multiple datasets by target code value.

*Note: The data identification process was iterative. As we proceeded with EDA, we discovered some columns were not needed and others that were needed. As we analyzed the data during the data cleaning process, we discovered that earnings are complex, often made up of multiple jobs.*

# Data Collection

The following data sources were used for this project. Data was imported into Google Drive from the below links and modified as needed to support this project.

The primary datasets for this project were initially taken from the Census' [Annual Social and Economic Supplement (ASEC)](https://www.census.gov/programs-surveys/saipe/guidance/model-input-data/cpsasec.html) of the Current Population Survey (CPS) for 2020. However, because we thought that 2020 might have been anomalous due to Covid, we chose to go back and take data from 2019 - pre-covid to get occupation and salary information that was more stable. Per the above link, the "*ASEC data is the source of timely official national estimates of poverty levels and rates and of widely used measures of income. It provides annual estimates based on a survey of more than 75,000 households. The survey contains detailed questions covering social and economic characteristics of each person who is a household member as of the interview date. Income questions refer to income received during the previous calendar year.*"

[Annual Social and Economic Survey (ASEC) All Years Data](https://www.census.gov/data/datasets/time-series/demo/cps/cps-asec.html)

* Contains links to all years from 2010 to 2021. CSV format was available from 2019 to 2021. Prior to 2019, a fixed format file was provided requiring an extra step to parse the data into a csv format.
* [2021 Survey - csv](https://www.census.gov/data/datasets/time-series/demo/cps/cps-asec.2021.html)
* [2020 Survey - csv](https://www.census.gov/data/datasets/time-series/demo/cps/cps-asec.2020.html)
* [2019 Survey - csv](https://www.census.gov/data/datasets/time-series/demo/cps/cps-asec.2019.html)
* [2018 Survey - dat](https://www.census.gov/data/datasets/time-series/demo/cps/cps-asec.2018.html) - Need to convert to csv

[Quarterly Census of Employment and Wages](https://www.bls.gov/cew/about-data/)

* Source data for OES Statistics. This was not used.

[Occupational Employment Wage Statistics (OES) Data](https://data.bls.gov/oes/#/geoOcc/Multiple%20occupations%20for%20one%20geographical%20area)

* Format - Excel converted to CSV
* Contained Occupational codes and aggregated statistics on wages for those occupations.
* We were not able to successfully merge this data with the Census data due to missing and mismatched State and SOC codes so removed this data from our final dataset.

[FIPS State Codes](https://www.census.gov/library/reference/code-lists/ansi/ansi-codes-for-states.html)

* Format - Copied from PDF and converted to CSV
* Contained FIPS State codes mapped to US Postal Service (USPS) State codes.
* Is used for data visualizations.

[Census Occupation Codes](https://www2.census.gov/programs-surveys/cps/techdocs/cpsmar20.pdf)

* Format - Copied from PDF and converted to CSV
* Contains Census Occupation codes mapped to Federal Standard Occupational Classification (SOC) Codes.
* Not used because the summarized occupation codes provided in the Census data were sufficient.

[Bureau of Labor Statistics SOC Codes](https://www.bls.gov/oes/current/oes_stru.htm#15-0000)

* Format - list on website
* Used to understand the Census occupation codes in more detail.

[Potential CNM Enrollment Data](https://www.cnm.edu/depts/finance-operations/ods/institutional-research-request)

* Not used.


## CNM Data Links - Not Used

We considered joining the CNM data with New Mexico respondent Census data. However CNM had restrictions placed on who could access their data that could not be resolved in the time available for this project. So CNM data was not used. 

[KPIs PDF](https://www.cnm.edu/depts/finance-operations/ods/dashboards-kpis/kpiupdate1120.pdf)

[CNM Data Dashboard](https://livecnm.sharepoint.com/sites/insights/FO/ODS/CNMdashboards/SitePages/Home.aspx?e=1%3Aa3207ba35cac40c480e6029dc0bfcc2c)

[CNM Graduate Outcomes](https://www.cnm.edu/depts/finance-operations/ods/documents/graduate-surveys/2018-2019-graduate-outcomes.pdf)

## Summarized Data Dictionary

The ASEC data dictionary is over 300 pages. Therefore, we created a summarized ASEC data dictionary containing only the variables that were used in this project.

[Here is a link to a summarized data dictionary.](https://docs.google.com/document/d/1io7TtqebJLtw6FKE7zkbUh26QkG3rEJrZX3Fver9zmU/edit)


# Imports

In [ ]:
# grab the imports needed for the project
import pandas as pd

# Globals
The team had different data links. The global here is to allow team members to specify who is working on this notebook so that they can run the code in their environment.

In [ ]:
# Expected values are: ellie, amy, cecilia - lowercase
team_member = 'cecilia'

# Root drive path
if team_member in ['amy','ellie']:
  root_drive = '/content/drive/MyDrive/'
else: # Cecilia
  root_drive = '/content/drive/MyDrive/Student Folder - Cecilia/Projects/'

# Data Cleaning

Once we identified the data elements needed for our project and the data sources that provided those data elements, the following steps were taken to get the data into a format needed for our analysis.

1. Downloaded data from data sources and placed copies in Google Drive.
1. Made changes to raw data, where needed, to support the project. 
  * Added State code to OES data and remove headers and footers from the data.
  * Created lookup data for State codes and SOC codes so secondary data sources could be merged with primary Census data. This involved cleaning the census code list so it could be properly parsed.
1. Converted codes in secondary datasets into Census codes.
1. Merged all datasets together into a single dataset.
1. Removed data that did not meet criteria for our analysis
  * Removed anyone under age 16.
  * Checked for nulls. 
1. Studied earnings/salary columns to determine which columns provided values that could be used for modeling. Added in columns that were missing from the initial analysis.
1. We were not able to reliably match the OES data to the census data using the full SOC Code because of disparities in SOC Codes. Therefore, we executed 3 matching passes reducing the SOC code by one character each time and pulling the largest Census code for the SOC code prefix. This allowed us to match a larger percentage of the data back to the ASEC data. However, we still had around 50% NULL data so decided to remove the OES data from our final analysis.
1. After initially applying models to our data, we decided to add additional predictor variables mostly from the family dataset indicating income source.
1. Because of poor model performance, we removed more variables and removed low-scoring occupation codes to balance the data. With these changes model performance increased from 17% to 50%.
1. With continuing average model performance, we decided to do some trend analysis for the past 10 years. To do this, we parsed 2018 to 2011 and merged it with the 2021-2019 data.

[GitHub link for data parsing on local system using Jupyter Notebooks through Anaconda Navigator](https://github.com/cfcastillo/DS-6-Notebooks/blob/main/Education%20Capstone%20Historic%20Data%20Parsing.ipynb)

[GitHub link for merging all years into a single dataset](https://github.com/cfcastillo/DS-6-Notebooks/blob/main/Education_Capstone_Historic_Data_Parsing_Server.ipynb)

This concluded our data cleaning and preparation steps.

## Import Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import Census data
asec_year = '19'
data_year = int('20' + asec_year)
asec_path = root_drive + 'Capstone/Data/ASEC/asecpub' + asec_year + 'csv/'
asec_data_person = pd.read_csv(asec_path + 'pppub' + asec_year + '.csv')
asec_data_household = pd.read_csv(asec_path + 'hhpub' + asec_year + '.csv')
asec_data_family = pd.read_csv(asec_path + 'ffpub' + asec_year + '.csv')

In [ ]:
# How many columns and rows do we have in each dataset?
print(f'Person data: {asec_data_person.shape}')
print(f'Household data: {asec_data_household.shape}')
print(f'Family data: {asec_data_family.shape}')

Person data: (180101, 799)
Household data: (94633, 135)
Family data: (79611, 87)


## ASEC Data

### Define ASEC Columns

[Here is a link to a summarized data dictionary.](https://docs.google.com/document/d/1io7TtqebJLtw6FKE7zkbUh26QkG3rEJrZX3Fver9zmU/edit)


In [ ]:
# Get lists of columns for various datasets that will be used for the project
# Note: Columns can be added as needed here and will propagate through the project.

# 12/4/2021 - Added continuous variables for better model performance - A_AGE, 
hid_col = ['H_IDNUM']
hseq_col = ['H_SEQ']
fseq_col = ['FH_SEQ']  # Joins to household data through H_SEQ
year_col = ['DATA_YEAR']
person_cols = ['OCCUP','POCCU2','A_MJOCC','A_DTOCC','AGE1','A_SEX','PRDTRACE','PXRACE1','PRCITSHP',
               'A_HGA','PRERELG', 'A_GRSWK', 'HRCHECK','HRSWK','PEARNVAL','A_CLSWKR','WEIND',
               'A_MARITL','A_HSCOL','A_WKSTAT','HEA','PEINUSYR', 'A_AGE']

# In 2022 data? - A_MAJACT, PURACEOT, RAC_HISP, UED_TYP

household_cols = ['GTMETSTA','GEDIV','GESTFIPS','HHINC','H_TENURE','H_LIVQRT']

# In 2022 data? - FEARNS, GEUR
# FKINDEX, 'FINC_ANN', 'FINC_DST', 'FINC_PEN' not in 2018 and earlier

family_cols = ['FINC_FR','FINC_SE','FINC_WS','FINC_CSP','FINC_DIS','FINC_DIV','FINC_RNT',
               'FINC_ED','FINC_SS','FINC_SSI','FINC_FIN','FINC_SUR','FINC_INT','FINC_UC',
               'FINC_OI','FINC_VET','FINC_PAW','FINC_WC']

### Get Household Id

In [ ]:
# Extract the Household id number from the person record so we can join the household and person dataframes by this id.
asec_data_person[hid_col] = asec_data_person['PERIDNUM'].str[:20]

In [ ]:
# View Person Data
asec_data_person[hid_col + person_cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180101 entries, 0 to 180100
Data columns (total 24 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   H_IDNUM   180101 non-null  object
 1   OCCUP     180101 non-null  int64 
 2   POCCU2    180101 non-null  int64 
 3   A_MJOCC   180101 non-null  int64 
 4   A_DTOCC   180101 non-null  int64 
 5   AGE1      180101 non-null  int64 
 6   A_SEX     180101 non-null  int64 
 7   PRDTRACE  180101 non-null  int64 
 8   PXRACE1   180101 non-null  int64 
 9   PRCITSHP  180101 non-null  int64 
 10  A_HGA     180101 non-null  int64 
 11  PRERELG   180101 non-null  int64 
 12  A_GRSWK   180101 non-null  int64 
 13  HRCHECK   180101 non-null  int64 
 14  HRSWK     180101 non-null  int64 
 15  PEARNVAL  180101 non-null  int64 
 16  A_CLSWKR  180101 non-null  int64 
 17  WEIND     180101 non-null  int64 
 18  A_MARITL  180101 non-null  int64 
 19  A_HSCOL   180101 non-null  int64 
 20  A_WKSTAT  180101 non-null 

In [ ]:
# Look at first 5 records of selected columns of person data.
asec_data_person[hid_col + person_cols].head()

,H_IDNUM,OCCUP,POCCU2,A_MJOCC,A_DTOCC,AGE1,A_SEX,PRDTRACE,PXRACE1,PRCITSHP,A_HGA,PRERELG,A_GRSWK,HRCHECK,HRSWK,PEARNVAL,A_CLSWKR,WEIND,A_MARITL,A_HSCOL,A_WKSTAT,HEA,PEINUSYR,A_AGE
0,01000691245394308011,4050,33,3,13,4,1,1,0,1,37,0,0,1,30,18000,1,18,7,0,4,3,0,21
1,39994039016100209011,0,53,0,0,17,2,1,0,1,39,0,0,0,0,0,0,23,4,0,1,3,0,85
2,91193400932060909011,4020,32,3,13,13,2,1,0,1,39,0,0,2,44,12000,1,18,7,0,2,3,0,61
3,14103203009699909011,0,53,0,0,16,2,1,0,1,39,0,0,0,0,0,0,23,5,0,1,5,0,73
4,14103203009699909011,4610,37,3,15,8,1,1,0,1,39,0,0,1,20,12000,1,16,7,0,4,3,0,37


In [ ]:
asec_data_person[hid_col + person_cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180101 entries, 0 to 180100
Data columns (total 24 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   H_IDNUM   180101 non-null  object
 1   OCCUP     180101 non-null  int64 
 2   POCCU2    180101 non-null  int64 
 3   A_MJOCC   180101 non-null  int64 
 4   A_DTOCC   180101 non-null  int64 
 5   AGE1      180101 non-null  int64 
 6   A_SEX     180101 non-null  int64 
 7   PRDTRACE  180101 non-null  int64 
 8   PXRACE1   180101 non-null  int64 
 9   PRCITSHP  180101 non-null  int64 
 10  A_HGA     180101 non-null  int64 
 11  PRERELG   180101 non-null  int64 
 12  A_GRSWK   180101 non-null  int64 
 13  HRCHECK   180101 non-null  int64 
 14  HRSWK     180101 non-null  int64 
 15  PEARNVAL  180101 non-null  int64 
 16  A_CLSWKR  180101 non-null  int64 
 17  WEIND     180101 non-null  int64 
 18  A_MARITL  180101 non-null  int64 
 19  A_HSCOL   180101 non-null  int64 
 20  A_WKSTAT  180101 non-null 

In [ ]:
# Convert H_IDNUM to object with left fill zero. Necessary for years 2018 and prior
if int(asec_year) <= 18:
  asec_data_household[hid_col[0]] = asec_data_household[hid_col[0]].astype(str).str.zfill(20)

In [ ]:
# View Household Data
asec_data_household[hid_col + hseq_col + household_cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94633 entries, 0 to 94632
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   H_IDNUM   94633 non-null  object
 1   H_SEQ     94633 non-null  int64 
 2   GTMETSTA  94633 non-null  int64 
 3   GEDIV     94633 non-null  int64 
 4   GESTFIPS  94633 non-null  int64 
 5   HHINC     94633 non-null  int64 
 6   H_TENURE  94633 non-null  int64 
 7   H_LIVQRT  94633 non-null  int64 
dtypes: int64(7), object(1)
memory usage: 5.8+ MB


In [ ]:
# Look at first 5 records of household data
asec_data_household[hid_col + hseq_col + household_cols].head()

,H_IDNUM,H_SEQ,GTMETSTA,GEDIV,GESTFIPS,HHINC,H_TENURE,H_LIVQRT
0,02031046575209908011,1,2,1,23,0,0,1
1,11000302225345308011,2,2,1,23,0,0,1
2,02031054530232108011,3,2,1,23,0,0,1
3,01000691245394308011,4,2,1,23,8,1,5
4,14320203005595208011,5,2,1,23,0,0,1


In [ ]:
# View family data. Get record count before grouping data.
asec_data_family[fseq_col + family_cols].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79611 entries, 0 to 79610
Data columns (total 19 columns):
 #   Column    Non-Null Count  Dtype
---  ------    --------------  -----
 0   FH_SEQ    79611 non-null  int64
 1   FINC_FR   79611 non-null  int64
 2   FINC_SE   79611 non-null  int64
 3   FINC_WS   79611 non-null  int64
 4   FINC_CSP  79611 non-null  int64
 5   FINC_DIS  79611 non-null  int64
 6   FINC_DIV  79611 non-null  int64
 7   FINC_RNT  79611 non-null  int64
 8   FINC_ED   79611 non-null  int64
 9   FINC_SS   79611 non-null  int64
 10  FINC_SSI  79611 non-null  int64
 11  FINC_FIN  79611 non-null  int64
 12  FINC_SUR  79611 non-null  int64
 13  FINC_INT  79611 non-null  int64
 14  FINC_UC   79611 non-null  int64
 15  FINC_OI   79611 non-null  int64
 16  FINC_VET  79611 non-null  int64
 17  FINC_PAW  79611 non-null  int64
 18  FINC_WC   79611 non-null  int64
dtypes: int64(19)
memory usage: 11.5 MB


In [ ]:
# There may be multiple families per household. We need unique records in order to merge
# with the household data.
asec_data_family_unique = asec_data_family.drop_duplicates(fseq_col + family_cols)[fseq_col + family_cols]

In [ ]:
# View family data after grouping. Get record count with all columns. 
# Compare with record count from sequence number column to ensure we truly have unique rows 
# and to see if further grouping is needed.
asec_data_family_unique

,FH_SEQ,FINC_FR,FINC_SE,FINC_WS,FINC_CSP,FINC_DIS,FINC_DIV,FINC_RNT,FINC_ED,FINC_SS,FINC_SSI,FINC_FIN,FINC_SUR,FINC_INT,FINC_UC,FINC_OI,FINC_VET,FINC_PAW,FINC_WC
0,4,2,2,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
1,6,2,2,2,2,2,2,2,2,1,2,2,2,2,2,2,2,2,2
2,7,2,2,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
3,8,2,2,1,2,2,2,2,2,1,1,2,2,2,2,2,2,2,2
4,13,2,2,2,2,2,2,2,2,1,2,2,2,1,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79605,94630,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
79606,94631,2,2,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
79608,94632,2,2,1,2,2,2,2,2,1,2,2,2,1,1,2,2,2,2
79609,94632,2,2,1,2,2,2,2,2,2,2,2,2,1,1,2,2,2,2


In [ ]:
# Looking at the unique results indicates that we still do not have a single family record per household
# because families within the household may have different income sources.  
# Therefore, we will take the first family record assuming it is the most significant record.  
# If extending this project in future, a better mechanism for reducing the data would be required.
temp_family = asec_data_family[asec_data_family['FFPOS'] == 1]
asec_data_family_single = temp_family[fseq_col + family_cols]

In [ ]:
temp_family.tail()

,GESTCEN,GTCBSA,FPOVCUT,FPERSONS,FHEADIDX,FSPOUIDX,FOWNU6,FRELU6,FKIND,FTYPE,FRELU18,FOWNU18,FLASTIDX,FMLASIDX,FH_SEQ,FAMLIS,FANNVAL,FCSPVAL,FDISVAL,FDIVVAL,FDSTVAL,FEARNVAL,FEDVAL,FFINVAL,FFPOS,FFRVAL,FHIP_VAL,FHIP_VAL2,FINC_ANN,FINC_CSP,FINC_DIS,FINC_DIV,FINC_DST,FINC_ED,FINC_FIN,FINC_FR,FINC_INT,FINC_OI,FINC_PAW,FINC_PEN,...,FINC_WC,FINC_WS,FINTVAL,FMED_VAL,FMOOP,FMOOP2,FOIVAL,FOTC_VAL,FOTHVAL,FPAWVAL,FPCTCUT,FPENVAL,FRECORD,FRNTVAL,FRSPOV,FRSPPCT,FSEVAL,FSPANISH,FSSIVAL,FSSVAL,FSUP_WGT,FSURVAL,FTOTVAL,FTOT_R,FUCVAL,FVETVAL,FWCVAL,FWSVAL,F_MV_FS,F_MV_SL,I_FHIPVAL,I_FHIPVAL2,I_FMEDVAL,I_FMOOP,I_FMOOP2,I_FOTCVAL,POVLL,FKINDEX,FILEDATE,MMYY
79603,95,46520,12043,1,1,0,0,0,2,2,0,0,1,1,94629,4,0,0,0,0,0,80000,0,0,1,0,0,4800,2,2,2,2,2,2,2,2,1,2,2,2,...,2,1,1,50,70,4870,0,20,20137,0,0,0,2,0,0,0,0,2,0,20136,44401,0,100137,41,0,0,0,80000,0,0,0,1,0,0,1,0,14,3,110419,32019
79604,95,46520,12043,1,1,0,0,0,3,2,0,0,1,1,94630,4,0,0,0,0,0,5040,0,0,1,0,0,0,2,2,2,2,2,2,2,2,1,2,2,2,...,2,1,9000,0,200,200,0,200,25801,0,0,0,2,1,0,0,0,2,0,16800,41247,0,30841,13,0,0,0,5040,0,0,1,1,1,1,1,0,9,4,110419,32019
79606,95,46520,29509,5,1,0,1,1,3,1,4,4,5,5,94631,1,0,0,0,0,0,11000,0,0,1,0,0,0,2,2,2,2,2,2,2,2,2,2,2,2,...,2,1,0,0,125,125,0,125,0,0,1,0,2,0,0,0,0,2,0,0,51555,0,11000,5,0,0,0,11000,1500,1880,0,0,0,0,0,0,1,4,110419,32019
79608,95,46520,35324,6,1,2,0,0,1,1,2,0,6,2,94632,4,0,0,0,0,0,58000,0,0,1,0,11808,11808,2,2,2,2,2,2,2,2,1,2,2,2,...,2,1,12,1270,13398,13398,0,320,9952,0,9,0,2,0,0,0,0,2,0,9240,39167,0,67952,28,700,0,0,58000,3600,1110,3,3,3,3,3,3,7,1,110419,32019
79610,95,46520,16815,2,1,0,0,0,3,1,0,0,2,2,94633,4,0,0,0,0,0,50000,0,0,1,0,0,0,2,2,2,2,2,2,2,2,2,2,2,2,...,2,1,0,9,309,309,0,300,0,0,7,0,2,0,0,0,35000,2,0,0,44829,0,50000,21,0,0,0,15000,0,0,3,3,3,3,3,3,9,4,110419,32019


In [ ]:
# View results
asec_data_family_single

,FH_SEQ,FINC_FR,FINC_SE,FINC_WS,FINC_CSP,FINC_DIS,FINC_DIV,FINC_RNT,FINC_ED,FINC_SS,FINC_SSI,FINC_FIN,FINC_SUR,FINC_INT,FINC_UC,FINC_OI,FINC_VET,FINC_PAW,FINC_WC
0,4,2,2,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
1,6,2,2,2,2,2,2,2,2,1,2,2,2,2,2,2,2,2,2
2,7,2,2,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
3,8,2,2,1,2,2,2,2,2,1,1,2,2,2,2,2,2,2,2
4,13,2,2,2,2,2,2,2,2,1,2,2,2,1,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79603,94629,2,2,1,2,2,2,2,2,1,2,2,2,1,2,2,2,2,2
79604,94630,2,2,1,2,2,2,1,2,1,2,2,2,1,2,2,2,2,2
79606,94631,2,2,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
79608,94632,2,2,1,2,2,2,2,2,1,2,2,2,1,1,2,2,2,2


### Merge ASEC Tables

In [ ]:
# Join Household and Personal records into single dataframe
# Inner join - should not have person without household.
asec_combined = pd.merge(asec_data_household[hid_col + hseq_col + household_cols], asec_data_person[hid_col + person_cols], on=hid_col)

# Join Family to get FINC columns
asec_combined = pd.merge(asec_combined, asec_data_family_single[fseq_col + family_cols], left_on=hseq_col, right_on=fseq_col, how='left')

# Add data year so that we can do trend analysis
asec_combined[year_col] = data_year

In [ ]:
# View combined results
asec_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180101 entries, 0 to 180100
Data columns (total 51 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   H_IDNUM    180101 non-null  object
 1   H_SEQ      180101 non-null  int64 
 2   GTMETSTA   180101 non-null  int64 
 3   GEDIV      180101 non-null  int64 
 4   GESTFIPS   180101 non-null  int64 
 5   HHINC      180101 non-null  int64 
 6   H_TENURE   180101 non-null  int64 
 7   H_LIVQRT   180101 non-null  int64 
 8   OCCUP      180101 non-null  int64 
 9   POCCU2     180101 non-null  int64 
 10  A_MJOCC    180101 non-null  int64 
 11  A_DTOCC    180101 non-null  int64 
 12  AGE1       180101 non-null  int64 
 13  A_SEX      180101 non-null  int64 
 14  PRDTRACE   180101 non-null  int64 
 15  PXRACE1    180101 non-null  int64 
 16  PRCITSHP   180101 non-null  int64 
 17  A_HGA      180101 non-null  int64 
 18  PRERELG    180101 non-null  int64 
 19  A_GRSWK    180101 non-null  int64 
 20  HRCH

In [ ]:
asec_combined.head()

,H_IDNUM,H_SEQ,GTMETSTA,GEDIV,GESTFIPS,HHINC,H_TENURE,H_LIVQRT,OCCUP,POCCU2,A_MJOCC,A_DTOCC,AGE1,A_SEX,PRDTRACE,PXRACE1,PRCITSHP,A_HGA,PRERELG,A_GRSWK,HRCHECK,HRSWK,PEARNVAL,A_CLSWKR,WEIND,A_MARITL,A_HSCOL,A_WKSTAT,HEA,PEINUSYR,A_AGE,FH_SEQ,FINC_FR,FINC_SE,FINC_WS,FINC_CSP,FINC_DIS,FINC_DIV,FINC_RNT,FINC_ED,FINC_SS,FINC_SSI,FINC_FIN,FINC_SUR,FINC_INT,FINC_UC,FINC_OI,FINC_VET,FINC_PAW,FINC_WC,DATA_YEAR
0,01000691245394308011,4,2,1,23,8,1,5,4050,33,3,13,4,1,1,0,1,37,0,0,1,30,18000,1,18,7,0,4,3,0,21,4,2,2,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2019
1,39994039016100209011,6,2,1,23,9,2,1,0,53,0,0,17,2,1,0,1,39,0,0,0,0,0,0,23,4,0,1,3,0,85,6,2,2,2,2,2,2,2,2,1,2,2,2,2,2,2,2,2,2,2019
2,91193400932060909011,7,2,1,23,5,2,1,4020,32,3,13,13,2,1,0,1,39,0,0,2,44,12000,1,18,7,0,2,3,0,61,7,2,2,1,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2019
3,14103203009699909011,8,2,1,23,10,2,1,0,53,0,0,16,2,1,0,1,39,0,0,0,0,0,0,23,5,0,1,5,0,73,8,2,2,1,2,2,2,2,2,1,1,2,2,2,2,2,2,2,2,2019
4,14103203009699909011,8,2,1,23,10,2,1,4610,37,3,15,8,1,1,0,1,39,0,0,1,20,12000,1,16,7,0,4,3,0,37,8,2,2,1,2,2,2,2,2,1,1,2,2,2,2,2,2,2,2,2019


## Combine All Data

In [ ]:
asec_final = asec_combined[year_col + household_cols + person_cols + family_cols]

In [ ]:
asec_final.shape

(180101, 48)

In [ ]:
# Review results of merged data
asec_final.info()
# asec_final.head(50)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180101 entries, 0 to 180100
Data columns (total 48 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   DATA_YEAR  180101 non-null  int64
 1   GTMETSTA   180101 non-null  int64
 2   GEDIV      180101 non-null  int64
 3   GESTFIPS   180101 non-null  int64
 4   HHINC      180101 non-null  int64
 5   H_TENURE   180101 non-null  int64
 6   H_LIVQRT   180101 non-null  int64
 7   OCCUP      180101 non-null  int64
 8   POCCU2     180101 non-null  int64
 9   A_MJOCC    180101 non-null  int64
 10  A_DTOCC    180101 non-null  int64
 11  AGE1       180101 non-null  int64
 12  A_SEX      180101 non-null  int64
 13  PRDTRACE   180101 non-null  int64
 14  PXRACE1    180101 non-null  int64
 15  PRCITSHP   180101 non-null  int64
 16  A_HGA      180101 non-null  int64
 17  PRERELG    180101 non-null  int64
 18  A_GRSWK    180101 non-null  int64
 19  HRCHECK    180101 non-null  int64
 20  HRSWK      180101 non-null

## Clean Data

In [ ]:
# Remove people under 15 years old because they are not relevant for this project.
# 0 = Not in universe
# 1 = 15 years
# 2 = 16 and 17 years
# 3 = 18 and 19 years
# 4 = 20 and 21 years
# 5 = 22 to 24 years
# 6 = 25 to 29 years
# 7 = 30 to 34 years
# 8 = 35 to 39 years
# 9 = 40 to 44 years
# 10 = 45 to 49 years
# 11 = 50 to 54 years
# 12 = 55 to 59 years
# 13 = 60 to 61 years
# 14 = 62 to 64 years
# 15 = 65 to 69 years
# 16 = 70 to 74 years
# 17 = 75 years and over
asec_final = asec_final[asec_final['AGE1'] > 0]
asec_final.info()
# asec_oes.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 141251 entries, 0 to 180100
Data columns (total 48 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   DATA_YEAR  141251 non-null  int64
 1   GTMETSTA   141251 non-null  int64
 2   GEDIV      141251 non-null  int64
 3   GESTFIPS   141251 non-null  int64
 4   HHINC      141251 non-null  int64
 5   H_TENURE   141251 non-null  int64
 6   H_LIVQRT   141251 non-null  int64
 7   OCCUP      141251 non-null  int64
 8   POCCU2     141251 non-null  int64
 9   A_MJOCC    141251 non-null  int64
 10  A_DTOCC    141251 non-null  int64
 11  AGE1       141251 non-null  int64
 12  A_SEX      141251 non-null  int64
 13  PRDTRACE   141251 non-null  int64
 14  PXRACE1    141251 non-null  int64
 15  PRCITSHP   141251 non-null  int64
 16  A_HGA      141251 non-null  int64
 17  PRERELG    141251 non-null  int64
 18  A_GRSWK    141251 non-null  int64
 19  HRCHECK    141251 non-null  int64
 20  HRSWK      141251 non-null

In [ ]:
asec_final.shape

(141251, 48)

## Export Data to CSV

Uncomment the below section when exporting files. Remember to change the version number if altering columns.

In [ ]:
# Export to CSV for teammates to use in EDA
# export_path = root_drive + 'Capstone/Data/FinalData/Trends/asec_' + str(data_year) + '_trend_v4.csv'
# asec_final.to_csv(export_path)

# File Versions
- V2 - Add DATA_YEAR and has data for 2012-2021
- V3 - Add POCCU2 - 53 occupational categories
- V4 - Add A_AGE - continuous age value

# Exploratory Data Analysis (EDA)

The EDA process can be found in the notebook titled [2. Education Capstone EDA.ipynb](https://colab.research.google.com/drive/1Fa18G_kZY8fCEKupjsfICRyeav7dEw7K)